In [3]:
from mistral_inference.transformer import Transformer

model = Transformer.from_folder("/home/shared_models/huggingface/mistralai/Mistral-Nemo-Instruct-2407")

/home/jonathan_lu/miniconda3/envs/mistral/lib/python3.11/site-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_fwd")
/home/jonathan_lu/miniconda3/envs/mistral/lib/python3.11/site-packages/xformers/ops/fmha/flash.py:344: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_bwd")


In [14]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
    ToolMessage,
    FinetuningAssistantMessage
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
    ToolCall,
    FunctionCall
)
from mistral_common.protocol.instruct.validator import (
    ValidationMode,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "mistral-nemo"

tokenizer = MistralTokenizer.from_file(str(MistralTokenizer._data_path() / "tekken_240718.json"), mode=ValidationMode.finetuning)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            SystemMessage(content="this is a system message"),
            UserMessage(content="What's the weather like today in Paris"),
            FinetuningAssistantMessage(content="Let me search that up for you"),
            FinetuningAssistantMessage(tool_calls=[ToolCall(id="3XhQnxLsT", function=FunctionCall(name="get_current_weather", arguments='{"location": "Paris, FR", "format": "celsius"}'))]),
            ToolMessage(tool_call_id="3XhQnxLsT", content="20"),
            FinetuningAssistantMessage(content="The weather is 20 degrees Celsius"),
            UserMessage(content="Describe what that temperature feels like"),
            FinetuningAssistantMessage(content="It feels warm"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

222


In [15]:
print(text)

<s>[INST]What's the weather like today in Paris[/INST]Let me search that up for you[TOOL_CALLS][{"name": "get_current_weather", "arguments": {"location": "Paris, FR", "format": "celsius"}, "id": "3XhQnxLsT"}]</s>[TOOL_RESULTS]{"content": 20, "call_id": "3XhQnxLsT"}[/TOOL_RESULTS]The weather is 20 degrees Celsius</s>[AVAILABLE_TOOLS][{"type": "function", "function": {"name": "get_current_weather", "description": "Get the current weather", "parameters": {"type": "object", "properties": {"location": {"type": "string", "description": "The city and state, e.g. San Francisco, CA"}, "format": {"type": "string", "enum": ["celsius", "fahrenheit"], "description": "The temperature unit to use. Infer this from the users location."}}, "required": ["location", "format"]}}}][/AVAILABLE_TOOLS][INST]Describe what that temperature feels like[/INST]It feels warm</s>


In [16]:
from mistral_inference.generate import generate

out_tokens, _ = generate([tokens], model, max_tokens=1024, temperature=0.35, eos_id=tokenizer.instruct_tokenizer.tokenizer.eos_id)
result = tokenizer.instruct_tokenizer.tokenizer.decode(out_tokens[0])

print(result)

Would you like to know the forecast for the next few days?
